# Tutorial on Boolean Model

<!-- SUMMARY: Perform conditional and non-conditional simulation with a variety of different object families -->

<!-- CATEGORY: Methodology -->

This file is meant to demonstrate the use of gstlearn for Boolean Model

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import matplotlib.pyplot as plt
import random as rnd

gdoc.setNoScroll()

Setting some global variables

In [ ]:
# Set the Global Options
verbose = True
flagGraphic = True

# Define the Space Dimension
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN, ndim)

# Set the Seed for the Random Number generator
gl.law_set_random_seed(5584)
rnd.seed(13155)

gp.setDefaultGeographic(dims=[7,7])

In [ ]:
nxcell = 100
grid = gl.DbGrid.create([nxcell,nxcell])
grid.display();

We now establish the stationary Boolean Model

In [ ]:
tokens = gl.ModelBoolean(0.01, True)

We must first define the dictionary of shapes used in the Model, composed of

- Ellipsoids
- parallelelipieds

In [ ]:
token_ellipsoid = gl.ShapeEllipsoid(0.4, 10., 20., 2.)
token_ellipsoid.setFactorX2Y(1.5)
tokens.addToken(token_ellipsoid)

token_parallelepiped = gl.ShapeParallelepiped(0.6, 5, 7., 1.)
tokens.addToken(token_parallelepiped)
tokens.display()

# Non conditional simulation

In [ ]:
err = gl.simbool(None, grid, tokens, namconv=gl.NamingConvention("NC"))
grid

We have created two new variables in the output Grid file:

- the Facies
- the Rank of the object

In [ ]:
ax = grid.plot(nameRaster="NC.Facies")
ax.decoration(title="Facies")

In [ ]:
ax = grid.plot(nameRaster="NC.Rank", flagLegendRaster=True)
ax.decoration(title="Rank")

# Conditional simulation

## Few conditioning points

We sample the grid in order to create a new Point data base: the number of selected points is small.

In [ ]:
ndat = 20
randomlist = rnd.sample(range(0, grid.getSampleNumber()), ndat)
data = gl.Db.createReduce(grid, ranks = randomlist)

In [ ]:
ax = grid.plot(nameRaster="NC.Facies")
ax = data.plot(nameColor="NC.Facies")
ax.decoration(title="Conditioning Information")

We must slightly transform the facies information that will serve as conditioning

In [ ]:
data["Facies1"] = np.nan_to_num(data["NC.Facies"], nan=0.)
data.setLocator("Facies1",gl.ELoc.Z, cleanSameLocator=True)

We can now launch a conditional simulation

In [ ]:
err = gl.simbool(data, grid, tokens, verbose=True, namconv=gl.NamingConvention("CD1"))
grid

In [ ]:
ax = grid.plot(nameRaster="CD1.Facies1.Facies")
ax = data.plot(nameColor="Facies1")
ax.decoration(title="Conditional Simulation (Small)")

## Lots of conditioning points

We now sample the grid in order to create a lot of conditioning samples

In [ ]:
ndat = 20
randomlist = rnd.sample(range(0, grid.getSampleNumber()), ndat)
data = gl.Db.createReduce(grid, ranks = randomlist)

In [ ]:
data["Facies2"] = np.nan_to_num(data["NC.Facies"], nan=0.)
data.setLocator("Facies2",gl.ELoc.Z, cleanSameLocator=True)

In [ ]:
ax = grid.plot(nameRaster="NC.Facies")
ax = data.plot(nameColor="NC.Facies")
ax.decoration(title="Conditioning Information (Large)")

In [ ]:
err = gl.simbool(data, grid, tokens, verbose=True, namconv=gl.NamingConvention("CD2"))
grid

In [ ]:
ax = grid.plot(nameRaster="CD2.Facies2.Facies")
ax = data.plot(nameColor="Facies2")
ax.decoration(title="Conditional Simulation (Large)")